<img style="float: right;" src="img/column.png" width="250">

<figure>
    <a href="http://www.upm.es">
    <img style="float: right;" src="img/UPMLogo.png" width="100">
    </a>
</figure>

# Destilación fraccionada
# Método de McCabe-Thiele

#### © **[Jorge Ramírez](http://blogs.upm.es/compsoftmatter/)**, **[Universidad Politécnica de Madrid](http://www.upm.es)**, 2019

En esta unidad vamos a repasar algunos conceptos básicos sobre presión de vapor y volatilidad de sustancias puras y mezclas binarias ideales. Después, utilizaremos esa información para resolver dos problemas de destilación que aparecen con frecuencia en Ingeniería Química:

- Destilación Flash
- Destilación fraccionada

Comenzamos cargando algunas de las librerías que utilizaremos en esta presentación.

In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from ipywidgets import interact, interactive, fixed, interact_manual, widget, widgets, Layout, HBox, VBox

## Presión de Vapor de sustancias puras: Ecuación de Antoine

La ecuación de Antoine es una correlación semi-empírica que describe la dependencia de la presión de vapor con la temperatura para líquidos puros. Está dada por la expresión:
$$ \log_{10} P=A-\frac{B}{C+T} $$

Donde $A$, $B$ y $C$ tienen valores constantes que dependen de la sustancia que se esté considerando. Típicamente, la ecuación de Antoine está definida para un determinado intervalo de temperatura $[T_1, T_2]$ y es importante no salirse de dicho intervalo. En esta presentación, no tendremos en cuenta los intervalos de validez.

Vamos a tomar de la base de datos de **[DDBST](http://ddbonline.ddbst.com/AntoineCalculation/AntoineCalculationCGI.exe)** los valores de las constantes de la ecuación de Antoine para una serie de sustancias. Para definir la ecuación de Antoine para cada sustancia, utilizamos un **diccionario** y definimos las funciones mediante la notación **lambda** (las temperaturas están dadas en $\circ$C y las presiones en mmHg):

In [3]:
# Create a dictionary with the Antoine Equation for each of the components
Psat = dict()
Psat['Acetona'] = lambda T: 10**(7.1327 - 1219.97/(T + 230.653))
Psat['Acetonitrilo'] = lambda T: 10**(7.33986 - 1482.29/(T + 250.523))
Psat['Acido Acetico'] = lambda T: 10**(7.2996 - 1479.02/(T + 216.82))
Psat['Agua'] = lambda T: 10**(8.07131 - 1730.63/(T + 233.426))
Psat['Etanol'] = lambda T: 10**( 8.20417 - 1642.89/(T + 230.3))
Psat['Etilenglicol'] = lambda T: 10**( 8.7945 - 2615.4/(T + 244.91))
Psat['Fenol'] = lambda T: 10**( 7.1345 - 1516.07/(T + 174.57))
Psat['isopropyl-alcohol'] = lambda T: 10**( 8.1182 - 1580.92/(T + 219.62))

A continuación, inspeccionamos cómo varía la presión de vapor para dos de las sustancias escogidas con respecto a la temperatura:

In [5]:
@interact(FluidA=Psat.keys(),FluidB=Psat.keys())
def check(FluidA='Acetona', FluidB='Agua'):
    T = np.linspace(200,350,100)
    plt.figure(2,figsize=(6, 4), dpi= 100)    
    plt.plot(T,Psat[FluidA](T),color='black',label=FluidA)
    plt.plot(T,Psat[FluidB](T),color='red',label=FluidB)
    plt.xlabel('T $\circ$C')
    plt.ylabel('$P^0$ [mmHg]')
    plt.legend()
    plt.minorticks_on()
    plt.grid(linewidth=1, which='both')
    plt.show()

interactive(children=(Dropdown(description='FluidA', options=('Acetona', 'Acetonitrilo', 'Acido Acetico', 'Agu…

A partir de la **presión de vapor**, se puede determinar la **temperatura de ebullición** de un fluido puro como aquella temperatura $T$ a la que su presión de vapor del fluido se hace igual a la presión exterior. Podemos buscar dicha solución mediante la función **fsolve**. Podemos comprobar la precisión de la ecuación de Antoine para el agua:

In [14]:
Tsat = dict()
for s in Psat.keys():
    Tsat[s] = lambda P, s=s: fsolve(lambda T: Psat[s](T)-P,50)[0]
    
print("Seleccione un fluido y especifique una presión en atmósferas:")
    
@interact(f=Tsat.keys(),P=(0.1,5.0))
def getTeb(f='Agua', P=1):
    print("Punto de ebullición del %s : %.3f"%(f,Tsat[f](P*760)))

Seleccione un fluido y especifique una presión en atmósferas:


interactive(children=(Dropdown(description='f', index=3, options=('Acetona', 'Acetonitrilo', 'Acido Acetico', …

También podemos comparar, de forma gráfica, las temperaturas de ebullición de dos fluidos diferentes en función de la Presión exterior (en atmósferas):

In [17]:
@interact(FluidA=Psat.keys(),FluidB=Psat.keys())
def check(FluidA='Acetona', FluidB='Agua'):
    P = np.linspace(100,5000,100)
    TA=np.zeros(len(P))
    TB=np.zeros(len(P))
    for i in range(len(P)):
        TA[i]=Tsat[FluidA](P[i])
        TB[i]=Tsat[FluidB](P[i])
    plt.figure(2,figsize=(6, 4), dpi= 100)    
    plt.plot(P/760,TA,color='black',label=FluidA)
    plt.plot(P/760,TB,color='red',label=FluidB)
    plt.xlabel('$P_{ext}$ [atm]')
    plt.ylabel('T [$\circ$C]')
    plt.legend()
    plt.minorticks_on()
    plt.grid(linewidth=1, which='both')
    plt.show()

interactive(children=(Dropdown(description='FluidA', options=('Acetona', 'Acetonitrilo', 'Acido Acetico', 'Agu…

## Mezclas binarias ideales: Ley de Raoult y diagrama P-x

La **ley de Raoult** supone que, en una **mezcla ideal** de dos líquidos, la presión de vapor de cada uno de los componentes de la mezcla es **proporcional** a su **fracción molar** dentro de la disolución:

$$P_i = P_i^0 x_i$$

Utilizando esta definición, podemos representar la presión de vapor de una mezcla binaria de dos sustancias A y B.

**NOTA**: En realidad, las mezclas no suelen ser ideales y suelen presentar desviaciones con respecto a la ley de Raoult.

In [18]:
@interact(FluidA=Psat.keys(),FluidB=Psat.keys(),T=(200,400))
def check(FluidA='Acetona', FluidB='Agua',T=298.15):
    plt.figure(2,figsize=(6, 4), dpi= 100)    
    PA=Psat[FluidA](T)
    PB=Psat[FluidB](T)
    x=np.linspace(0,1,100)
    plt.plot(x,PA*x,color='blue',label="$P^0_{"+FluidA+"}$")
    plt.plot(x,PB*(1-x),color='red',label="$P^0_{"+FluidB+"}$")
    plt.plot(x,PB*(1-x)+PA*x,color='black',label="$P^0_{TOTAL}$")
    plt.xlabel("$x_{"+FluidA+"}$")
    plt.ylabel('$P$ [mmHg]')
    plt.legend()
    plt.minorticks_on()
    plt.grid(linewidth=1, which='both')
    plt.xlim(0,1)
    plt.ylim(0,1.1*max(PA,PB))
    plt.show()

interactive(children=(Dropdown(description='FluidA', options=('Acetona', 'Acetonitrilo', 'Acido Acetico', 'Agu…

## Diagramas T-x: Destilación Flash

<img style="float: right;" src="img/Vap-Liq_Separator.png" width="200">

La **destilación Flash** es una **operación unitaria** en la que una **mezcla líquida saturada** se introduce en un tanque a **menor presión**, en el que se separa en dos fases, un vapor y un líquido enriquecidos en el componente más volátil y menos volátil, respectivamente. La presión $P$ y temperatura $T$ dentro del tanque se deben elegir cuidadosamente para optimizar la separación.

El punto de ebullición de la mezcla líquida binaria será la temperatura a la que la presión de vapor de la mezcla se haga igual a la presión exterior. 

$$ P_{ext} = P_A^0(T)x_A + P_B^0(T)(1-x_A) $$

Si conocemos la temperatura $T$, podemos determinar la composición de la mecla que tendrá dicha temperatura de ebullición. Despejando $x_A$ de la ecuación anterior:
$$ x_A = \frac{P_{ext}-P_B^0(T)}{P_A^0(T)-P_B^0(T)},$$
así comola composición del vapor que se desprenderá a esa temperatura:
$$ y_A = \frac{x_A P_A^0(T)}{P_{ext}}$$

Primero, vamos a explorar qué forma tienen los diagramas **T-x** (temperatura-composición) para mezclas ideales de fluidos, en función de la presión exterior $P$ y de la composición de la mezcla.

In [25]:
@interact(FluidA=Psat.keys(),FluidB=Psat.keys(),Pext=(0.2,5.0))
def check(FluidA='Acetona', FluidB='Agua',Pext=1.0):
    P=Pext*760
    x = lambda T: (P-Psat[FluidB](T))/(Psat[FluidA](T)-Psat[FluidB](T))
    y = lambda T: x(T)*Psat[FluidA](T)/P
    plt.figure(2,figsize=(6, 4), dpi= 100)
    T = np.linspace(Tsat[FluidA](P),Tsat[FluidB](P))
    plt.plot([x(T) for T in T],T,color='black')
    plt.plot([y(T) for T in T],T,color='black')
    plt.xlabel("$x_{"+FluidA+"}$")
    plt.ylabel('Temperatura $^\circ$C')
    plt.title('Diagrama Txy para {:s}/{:s} a P = {:.0f} atm'.format(FluidA,FluidB,P))
    Tmid=0.5*(Tsat[FluidA](P)+Tsat[FluidB](P))
    plt.text(x(Tmid),Tmid-5,'L',size=15)
    plt.text(y(Tmid),Tmid+5,'V',size=15)
    plt.minorticks_on()
    plt.grid(linewidth=1, which='both')
    plt.xlim(0,1)
    plt.show()

interactive(children=(Dropdown(description='FluidA', options=('Acetona', 'Acetonitrilo', 'Acido Acetico', 'Agu…

Con esa información, vamos a intentar **resolver un problema de destilación Flash** en el que conocemos: 

- Los dos fluidos A y B (intentaremos seleccionar A como el más volátil)
- La composición $x_F$ y caudal $F$ de la mezcla que se alimenta al destilador.
- La Temperatura $T$ y presión $P$ dentro del destilador.

In [35]:
@interact(FluidA=Psat.keys(),FluidB=Psat.keys(),Pext=(0.2,5.0),xF=(0.0,1.0),TF=(0.0,300.0),F=(20,200))
def check(FluidA='Acetona', FluidB='Agua',Pext=1.0,xF=0.5,TF=100.0,F=100.0):
    P=Pext*760
    x = lambda T: (P-Psat[FluidB](T))/(Psat[FluidA](T)-Psat[FluidB](T))
    y = lambda T: x(T)*Psat[FluidA](T)/P
    plt.figure(2,figsize=(6, 4), dpi= 100)
    T = np.linspace(Tsat[FluidA](P),Tsat[FluidB](P))
    plt.plot([x(T) for T in T],T,color='black')
    plt.plot([y(T) for T in T],T,color='black')
    plt.xlabel("$x_{"+FluidA+"}$")
    plt.ylabel('Temperatura $^\circ$C')
    plt.title('Diagrama Txy para {:s}/{:s} a P = {:.1f} atm'.format(FluidA,FluidB,Pext))
    plt.minorticks_on()
    plt.grid(linewidth=1, which='both')
    plt.xlim(0,1)

    Tdew = fsolve(lambda T: y(T)-xF, 138)
    Tbub = fsolve(lambda T: x(T)-xF, 0.01)
    
    ax = plt.axis()
    plt.plot(xF,TF,'kx',ms=8)
    plt.plot([xF,xF,0],[ax[2],TF,TF],'b--')
    plt.text(xF,ax[2],'$x_F$',size=20)
    plt.text(0.01,TF+1,'$T_F$',size=15)
    
    plt.plot(xF,Tdew,'kD',ms=8)
    plt.plot(xF,Tbub,'kD',ms=8)
    
    if (TF>=Tbub and TF<=Tdew):
        plt.plot(y(TF),TF,'go',ms=7)
        plt.plot([xF,y(TF),y(TF)],[TF,TF,ax[2]],'g--')
        plt.text(y(TF),ax[2],'$y=${:.2f}'.format(y(TF)),size=15)
        plt.plot(x(TF),TF,'ro',ms=7)
        plt.plot([xF,x(TF),x(TF)],[TF,TF,ax[2]],'r--')
        plt.text(x(TF),ax[2],'$x=${:.2f}'.format(x(TF)),size=15,horizontalalignment='right')
    plt.show()
    
    L=F*(y(TF)-xF)/(y(TF)-x(TF))
    V=F-L
    print("L = %.3f - V = %.3f"%(L, V))
    print("Tburbuja  {:.1f} °C - Trocio {:.1f} °C".format(Tbub[0],Tdew[0]))

interactive(children=(Dropdown(description='FluidA', options=('Acetona', 'Acetonitrilo', 'Acido Acetico', 'Agu…

<img style="float: right;" src="img/column.png" width="250">

## Diagramas y-x: Destilación Fraccionada - Metodo de McCabe-Thiele

Mediante destilación Flash, no es posible la separación completa de los componentes de una mezcla ideal. Para ello, es necesario hacer uso de la destilación fraccionada. 

En este tipo de operación, es más práctico trabajar con el diagrama **y-x** (composición del vapor frente a composición del líquido, con respecto al componente más volátil). 

Podemos explorar qué pinta tienen estos diagramas **y-x** para pares de fluidos de nuestra biblioteca (seleccionar el fluido **más volátil** como el **FluidoA**), en función de la presión a la que trabaja la columna.

In [39]:
@interact(FluidA=Psat.keys(),FluidB=Psat.keys(),Pext=(0.2,5.0))
def check(FluidA='Acetona', FluidB='Agua',Pext=1.0):
    P=Pext*760
    x = lambda T: (P-Psat[FluidB](T))/(Psat[FluidA](T)-Psat[FluidB](T))
    y = lambda T: x(T)*Psat[FluidA](T)/P
    T = np.linspace(Tsat[FluidA](P),Tsat[FluidB](P))
    plt.figure(figsize=(5,5),dpi=100)
    plt.plot([x(T) for T in T],[y(T) for T in T], color='black')
    plt.plot([0,1],[0,1],color='black',linestyle='--')
    plt.axis('equal')

    plt.title('Diagrama x-y para {:s}/{:s} a P = {:.1f} atm'.format(FluidA,FluidB,Pext))
    plt.xlabel("$x_{"+FluidA+"}$")
    plt.ylabel("$y_{"+FluidA+"}$")

    plt.xlim(0,1)
    plt.ylim(0,1)
    plt.minorticks_on()
    plt.grid(linewidth=1, which='both')


interactive(children=(Dropdown(description='FluidA', options=('Acetona', 'Acetonitrilo', 'Acido Acetico', 'Agu…

### Método de McCabe y Thiele

Se desea separar los componentes de una mezcla binaria ideal de composición $x_F$ (en el componente más volátil) en dos corrientes, una enriquecida en el componente más volátil ($x_D$) y otra empobrecida (de composición $x_B$).
El método de McCabe y Thiele es un método gráfico que permite diseñar y dimensionar operaciones de destilación fraccionada de forma sencilla, intuitiva e instructiva. El método precisa que tracemos tres rectas en el diagrama **y-x**:

- La **recta de alimentación**: pasa por el punto $(x_F, x_F)$ y la pendiente depende del estado en el que entra la alimentación a la columna (como líquido saturado, líquido subenfriado, vapor saturado, etc). 

- La **recta de operación de la zona de rectificación**: pasa por el punto  $(x_D, x_D)$ y proporciona la relación entre las composiciones del líquido que desciende desde un plato superior y la del vapor que asciende desde un plato inferior, en la zona de la columna por encima de la alimentación.

- La **recta de operación de la zona de agotamiento**: pasa por el punto  $(x_B, x_B)$, igual que la anterior pero para la zona de la columna por debajo de la alimentación.

Para trazar la recta de alimentación, necesitamos saber la $T_{burbuja}$, $T_{rocio}$, $C_p$(L) y $C_p$(V) de cada fluido puro y $\Delta H_{vap}$ de cada fluido puro.

In [40]:
Hvap = dict()
Hvap['Acetona'] = 31300
Hvap['Acetonitrilo'] = 33225
Hvap['Acido Acetico'] = 23700
Hvap['Agua'] = 40660
Hvap['Etanol'] = 38600
Hvap['Etilenglicol'] = 53200
Hvap['Fenol'] = 58800
Hvap['isopropyl-alcohol'] = 44000

cPLiq = dict()
cPLiq['Acetona'] = 125.5
cPLiq['Acetonitrilo'] = 91.7
cPLiq['Acido Acetico'] = 123.1
cPLiq['Agua'] = 75.327
cPLiq['Etanol'] = 112.4
cPLiq['Etilenglicol'] = 37.8
cPLiq['Fenol'] = 138.0
cPLiq['isopropyl-alcohol'] = 160.8 

cPVap = dict()
cPVap['Acetona'] = 75
cPVap['Acetonitrilo'] = 80 
cPVap['Acido Acetico'] = 63.4
cPVap['Agua'] = 37.47
cPVap['Etanol'] = 82.0
cPVap['Etilenglicol'] = 23.0
cPVap['Fenol'] = 145.52
cPVap['isopropyl-alcohol'] = 92.4


Una vez establecida la especificación, podemos explorar cómo afectan los diferentes parámetros de la destilación al número de etapas necesarias para llevarla a cabo. Para ello, hay que seleccionar:

- Fluidos que queremos separar (seleccione el **FluidoA** como el **más volátil** de los dos).
- Especificación de la operación (composición de la alimentación $x_F$, composición de la corriente de destilado $x_D$ y de la de fondo $x_B$).
- Relación de reflujo externa $R_{ext}$ a utilizar.
- Temperatura a la que entra la alimentacion ($T_F$).
- Presión a la que trabaja la columna ($P$).

In [43]:
def check(FluidA='Acetona', FluidB='Agua',Pext=1.0,TF=100.0,xF=0.5,xB=0.1,xD=0.9,Rext=3.0):
    P=Pext*760
    x = lambda T: (P-Psat[FluidB](T))/(Psat[FluidA](T)-Psat[FluidB](T))
    y = lambda T: x(T)*Psat[FluidA](T)/P
    T = np.linspace(Tsat[FluidA](P),Tsat[FluidB](P))
    plt.figure(figsize=(5,5),dpi=100)
    xT=x(T)
    yT=y(T)
    p=xT.argsort()
    xT=xT[p]
    yT=yT[p]
    plt.plot(xT,yT, color='black')
    plt.plot([0,1],[0,1],color='black',linestyle='--')
    plt.axis('equal')

    # RECTA Q
    Tdew = fsolve(lambda T: y(T)-xF, 138)[0]
    Tbub = fsolve(lambda T: x(T)-xF, 0.01)[0]
    
    Hvapmix=xF*Hvap[FluidA]+(1-xF)*Hvap[FluidB]
    cPVapmix=xF*cPVap[FluidA]+(1-xF)*cPVap[FluidB]
    cPLiqmix=xF*cPLiq[FluidA]+(1-xF)*cPLiq[FluidB]
    
    if (TF>Tdew):
        q = -(TF-Tdew)*(cPVapmix)/Hvapmix
    elif (TF>Tbub):
        q = (Tdew-TF)/(Tdew-Tbub)
    else:
        q = (Hvapmix+(Tbub-TF)*cPLiqmix)/Hvapmix
    
    yq=-xT*q/(1-q)+xF/(1-q)

    #Intersection occurs between points itemindex and itemindex+1
    itemindex = np.argwhere(np.diff(np.sign(yq - yT))).flatten()
    a1=np.array([xT[itemindex][0],yq[itemindex][0]])
    a2=np.array([xT[itemindex+1][0],yq[itemindex+1][0]])
    b1=np.array([xT[itemindex][0],yT[itemindex][0]])
    b2=np.array([xT[itemindex+1][0],yT[itemindex+1][0]])
    da = a2-a1
    db = b2-b1
    dp = a1-b1
    dap = np.empty_like(da)
    dap[0] = -da[1]
    dap[1] = da[0]
    denom = np.dot( dap, db)
    num = np.dot( dap, dp )
    intersec=(num / denom.astype(float))*db + b1    
    plt.plot([xF,intersec[0]],[xF,intersec[1]],color='blue')
    plt.plot(intersec[0],intersec[1],'ko',ms=5)

    # RECTA DE OPERACION DE ALIMENTACION (REFLUJO MINIMO)
    plt.plot([xD, intersec[0]], [xD,intersec[1]], 'c--')
    Rextmin=(intersec[1]-xD)/(intersec[0]-intersec[1])
    
    if (Rext<Rextmin):
        print('Reflujo menor que el mínimo permitido (%.2f)'%Rextmin)
        return
    
    # RECTA DE OPERACION DE ALIMENTACION
    plt.plot([xD,0],[xD,xD/(1+Rext)],'c-')
    
    # INTERSECCION RECTA DE OPERACION - RECTA DE ALIMENTACION
    a1=np.array([xF,xF])
    a2=intersec
    b1=np.array([xD,xD])
    b2=np.array([0,xD/(1+Rext)])
    da = a2-a1
    db = b2-b1
    dp = a1-b1
    dap = np.empty_like(da)
    dap[0] = -da[1]
    dap[1] = da[0]
    denom = np.dot( dap, db)
    num = np.dot( dap, dp )
    intersec2=(num / denom.astype(float))*db + b1    
    plt.plot(intersec2[0],intersec2[1],'mo',ms=5)
    
    # RECTA DE OPERACION - ZONA DE AGOTAMIENTO
    plt.plot([xB,intersec2[0]],[xB,intersec2[1]])
    S = (intersec2[1]-xB)/(intersec2[0]-xB)

    plt.title('Diagrama x-y para {:s}/{:s} a P = {:.1f} atm'.format(FluidA,FluidB,Pext))
    plt.xlabel("$x_{"+FluidA+"}$")
    plt.ylabel("$y_{"+FluidA+"}$")

    plt.plot(xF,xF,'go',ms=5)
    plt.plot(xD,xD,'bo',ms=5)
    plt.plot(xB,xB,'yo',ms=5)

    # START McCabe-Thiele
    xP = xD
    yP = xD
    nTray = 0

    while xP > xB:
        nTray += 1
        Tdew = fsolve(lambda T:y(T) - yP, 100)
        xQ = xP
        xP = x(Tdew)
        plt.plot([xQ,xP],[yP,yP],'r')
        #plt.plot(xP,yP,'ro',ms=5)
        plt.text(xP-0.03,yP,nTray)

        yQ = yP
        yP = min([xD - (Rext/(Rext+1))*(xD-xP),xB + S*(xP-xB)])
        plt.plot([xP,xP],[yQ,yP],'r')
        
        plt.plot([xB,xB],[0,xB],'b--')
        plt.plot([xF,xF],[0,xF],'b--')
        plt.plot([xD,xD],[0,xD],'b--')
        plt.text(xB,0.02,'$x_B$ = {:0.2f}'.format(float(xB)),horizontalalignment='center')
        plt.text(xF,0.02,'$x_F$ = {:0.2f}'.format(float(xF)),horizontalalignment='center')
        plt.text(xD,0.02,'$x_D$ = {:0.2f}'.format(float(xD)),horizontalalignment='center')
    
    plt.text(0.05,0.95,'{:d} etapas eq.'.format(int(nTray)),weight='bold')

    plt.xlim(0,1)
    plt.ylim(0,1)
    plt.minorticks_on()
    plt.grid(linewidth=1, which='both')

    
widget=interactive(check,FluidA=Psat.keys(),FluidB=Psat.keys(),
          Pext=widgets.FloatSlider(value=1.0,min=0.2,max=5.0,step=0.1), 
          TF=widgets.FloatSlider(value=100,min=30,max=300,step=5), 
          xF=widgets.FloatSlider(value=0.5,min=0.1,max=0.9,step=0.01), 
          xB=widgets.FloatSlider(value=0.1,min=0.01,max=0.49,step=0.01), 
          xD=widgets.FloatSlider(value=0.9,min=0.51,max=0.99,step=0.01), 
          Rext=widgets.FloatSlider(value=3.0,min=0.1,max=5.0,step=0.1))
controls = HBox(widget.children[:-1], layout = Layout(flex_flow='row wrap'))
output = widget.children[-1]
display(VBox([controls, output]))

## Bibliografía - Otros Recursos

- W.L. McCabe, J.C. Smith y P. Harriott, *Operaciones Unitarias en Ingeniería Química*, 7ª ed., McGraw Hill, 2007
- J. Kantor, *[Introduction to Chemical Engineering Analysis](https://github.com/jckantor/CBE20255)*, curso CBE 20255, Universidad de Notre Dame, Indiana, EE.UU.
- [LearnChemE](http://www.learncheme.com/simulations)